In [1]:
# !pip install scikit-learn --upgrade

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from google.colab import files

# uploaded = files.upload()


In [3]:

pd.set_option("display.max_columns", None)
datas = pd.read_csv("Pélec_08_conso_train.csv")
datas.head()

OSEBuildingID PrimaryPropertyType  CouncilDistrictCode  YearBuilt  \
0            1.0               hotel                  7.0     1927.0   
1            2.0               hotel                  7.0     1996.0   
2            3.0               hotel                  7.0     1969.0   
3            8.0               hotel                  7.0     1980.0   
4            9.0               other                  7.0     1999.0   

   NumberofBuildings  NumberofFloors  PropertyGFATotal  PropertyGFAParking  \
0                1.0            12.0           88434.0                 0.0   
1                1.0            11.0          103566.0             15064.0   
2                1.0            41.0          961990.0                 0.0   
3                1.0            18.0          119890.0             12460.0   
4                1.0             2.0           97288.0             37198.0   

   PropertyGFABuildings  ENERGYSTARScore  SiteEUIkBtusf  SiteEUIWNkBtusf  \
0               88434.0             65.0           78.9             80.3   
1               88502.0             51.0           94.4             99.0   
2              961990.0             18.0           96.6             99.7   
3              107430.0             67.0          120.1            122.1   
4               60090.0              0.0          135.7            146.9   

   SourceEUIkBtusf  SourceEUIWNkBtusf  SiteEnergyUsekBtu  SiteEnergyUseWNkBtu  \
0            173.5              175.1          6981428.0            7097539.0   
1            191.3              195.2          8354235.0            8765788.0   
2            242.7              246.5         73130656.0           75506272.0   
3            228.8              227.1         14829099.0           15078243.0   
4            313.5              321.6         12051984.0           13045258.0   

  Outlier  TotalGHGEmissions  GHGEmissionsIntensity   GFAperFloor  \
0     NaN             249.43                   2.64   7369.500000   
1     NaN             263.51                   2.38   8045.636364   
2     NaN            2061.48                   1.92  37005.500000   
3     NaN             507.70                   4.02  23463.170732   
4     NaN             304.62                   2.81  12171.272727   

   GFAperBuilding  HasParking  percGFABuilding  percGFAParking  \
0         88434.0         0.0         1.000000        0.000000   
1         88502.0         1.0         0.854547        0.145453   
2        148022.0         1.0         0.912466        0.087534   
3        961990.0         0.0         1.000000        0.000000   
4        133884.0         1.0         0.874128        0.125872   

   SiteEUIkBtusf_diff1516  SiteEUIWNkBtusf_diff1516  SourceEUIkBtusf_diff1516  \
0                2.799997                  4.000003                  9.000000   
1                2.550002                  0.950000                  3.249998   
2                2.550002                  0.950000                  3.249998   
3                6.833333                  6.366667                 11.100001   
4                6.833333                  6.366667                 11.100001   

   SourceEUIWNkBtusf_diff1516  SiteEnergyUsekBtu_diff1516  \
0                   13.900000               244934.500000   
1                    2.300000               168939.000000   
2                    2.300000               168939.000000   
3                   11.233335               539703.666667   
4                   11.233335               539703.666667   

   SiteEnergyUseWNkBtu_diff1516  SteamUsekBtu_diff1516  \
0                  4.754820e+05           0.000000e+00   
1                  4.194402e+05           0.000000e+00   
2                  4.194402e+05           0.000000e+00   
3                  1.018618e+06           1.173258e+06   
4                  1.018618e+06           1.173258e+06   

   ElectricitykBtu_diff1516  NaturalGaskBtu_diff1516  SteamperGFA  ElecperGFA  \
0             259867.000000                   4065.0    22.87

## classement des variables

* Les variables nominatives seront transformées par OneHotEncoder
* Les variables ordinales seront transformées par OrdinalEncoder
* Les variables numériques seront normalisées par QuantileTransformer

In [4]:
nom_features = ["PrimaryPropertyType", "CouncilDistrictCode", "Outlier" 
    ]
ord_features = ["YearBuilt", "NumberofFloors", "NumberofBuildings", "HasParking" 
    ]
num_features = ["PropertyGFATotal", "PropertyGFABuildings", "PropertyGFAParking", 
    "GFAperFloor", "GFAperBuilding", "percGFABuilding", "percGFAParking", 

    # "SiteEUIkBtusf_diff1516", "SiteEUIWNkBtusf_diff1516", "SourceEUIkBtusf_diff1516", "SourceEUIWNkBtusf_diff1516", 
    # "SiteEnergyUsekBtu_diff1516", "SiteEnergyUseWNkBtu_diff1516", 
    # "SteamUsekBtu_diff1516", "ElectricitykBtu_diff1516", "NaturalGaskBtu_diff1516", 
    
    "SteamperGFA", "ElecperGFA", "GasperGFA", "FuelperGFA", "percSteam", "percElec", "percGas", "percFuel", 
    "WNfactor", "SteamWN", "ElecWN", "GasWN", "FuelWN", "SteamWNperGFA", "ElecWNperGFA", "GasWNperGFA", "FuelWNperGFA", 
    "percSteamWN", "percElecWN", "percGasWN", "percFuelWN"
    ]
a_exclure = ["SiteEUIkBtusf", "SiteEUIWNkBtusf", "SourceEUIkBtusf", "SourceEUIWNkBtusf", 
    "SteamUsekBtu", "ElectricitykBtu", "NaturalGaskBtu", "OtherFuelUsekBtu", 
    "SiteEnergyUseWNkBtu", "TotalGHGEmissions", "GHGEmissionsIntensity"]
cible = ["SiteEnergyUsekBtu"]

In [5]:
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor

X = datas[nom_features + ord_features + num_features]
y = datas[cible]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

ct = ColumnTransformer([("onehotencoder", OneHotEncoder(handle_unknown="ignore"), nom_features),
    ("quantiletransformer", QuantileTransformer(output_distribution="normal"), num_features),
    ("ordinalencoder", OrdinalEncoder(handle_unknown= "use_encoded_value", unknown_value= -1), ord_features)
    ])
ct.fit(X_train)

X_train_ct = pd.DataFrame(ct.transform(X_train), columns= list(ct.transformers_[0][1].get_feature_names(nom_features)) + 
    num_features + ord_features)
X_test_ct = pd.DataFrame(ct.transform(X_test), columns= list(ct.transformers_[0][1].get_feature_names(nom_features)) + num_features
    + ord_features)

In [6]:
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [7]:
from sklearn.feature_selection import RFECV

ttr = TransformedTargetRegressor(xgboost.XGBRegressor(objective= "reg:squarederror"), 
    transformer=QuantileTransformer(output_distribution="normal"))

rfe = RFECV(ttr, cv= 5, scoring= "r2", n_jobs= -1, importance_getter= "regressor_.feature_importances_")
rfe.fit(X_train_ct, y_train)


RFECV(cv=5,
      estimator=TransformedTargetRegressor(regressor=XGBRegressor(base_score=None,
                                                                  booster=None,
                                                                  colsample_bylevel=None,
                                                                  colsample_bynode=None,
                                                                  colsample_bytree=None,
                                                                  gamma=None,
                                                                  gpu_id=None,
                                                                  importance_type='gain',
                                                                  interaction_constraints=None,
                                                                  learning_rate=None,
                                                                  max_delta_step=None,
                                         

In [8]:
print(f"R2 train: {rfe.score(X_train_ct, y_train)}")
print(f"R2 test: {rfe.score(X_test_ct, y_test)}")


R2 train: 0.9978117084896
R2 test: 0.6962634527171216


In [9]:
print(X_train_ct.columns[rfe.support_])

Index(['PrimaryPropertyType_college/university',
       'PrimaryPropertyType_distributioncenter',
       'PrimaryPropertyType_hospital', 'PrimaryPropertyType_hotel',
       'PrimaryPropertyType_k-12school', 'PrimaryPropertyType_largeoffice',
       'PrimaryPropertyType_medicaloffice',
       'PrimaryPropertyType_mixeduseproperty',
       'PrimaryPropertyType_non-refrigeratedwarehouse',
       'PrimaryPropertyType_other',
       'PrimaryPropertyType_refrigeratedwarehouse',
       'PrimaryPropertyType_restaurant', 'PrimaryPropertyType_retailstore',
       'PrimaryPropertyType_self-storagefacility',
       'PrimaryPropertyType_seniorcarecommunity',
       'PrimaryPropertyType_small-andmid-sizedoffice',
       'PrimaryPropertyType_supermarket/grocerystore',
       'PrimaryPropertyType_warehouse', 'PrimaryPropertyType_worshipfacility',
       'CouncilDistrictCode_1.0', 'CouncilDistrictCode_2.0',
       'CouncilDistrictCode_3.0', 'CouncilDistrictCode_4.0',
       'CouncilDistrictCode_5.0', '

In [9]:
pd.DataFrame(X_train_ct.columns[rfe.support_], columns=["columns"]).to_csv("Pélec_10_relevant_features.csv", index=False)
pd.DataFrame([], columns=["Ridge", "SVR", "LR", "Lasso", "XGB", "RFR"], index=["train_score", "test_score"]).to_csv("Pélec_10_scores.csv", 
    index=True)

In [10]:
# files.download("Pélec_10_relevant_features.csv")
# files.download("Pélec_10_scores.csv")